In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import fasttext
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [6]:
def load_data(file_path):
    df = pd.read_json(file_path, lines = True)
    df['category'] = pd.Categorical(df['category'])
    df = df[df["short_description"] != ""]
    df['headlineDesc'] = df['headline']+ ' ' + df['short_description']
    return df[["headlineDesc", "category"]]

df = load_data('Data/News_Category_Dataset_v2.json')
df_orig = df
df.head()

,headlineDesc,category
0,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,ENTERTAINMENT
4,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT


In [7]:
# Preprocessing
df = df_orig
column_names = ["category", "headlineDesc"]
df = df.reindex(columns=column_names)
num_records = df.size
print(num_records)
def shuffle_data(data):
    data = shuffle(data)
    print(data.head())
    data.reset_index(inplace=True, drop=True)
    return data
    
def fasttext_format(df_basic, per_train=0.7):
    def take_percent_rows(data, perc=0.7, headortail="head"):
        if(headortail=="head"):
            return data.head(int(len(data)*(perc)))
        else:
            return data.tail(int(len(data)*(perc)))
    def format_text(data):
        data['category_formatted'] ='__label__' + data['category'].astype(str)
        return data[['category_formatted', 'headlineDesc']]
        
    df_train = take_percent_rows(df_basic, perc=per_train, headortail="head")
    df_valid = take_percent_rows(df_basic, perc=(1-per_train), headortail="tail")
    
    df_train = format_text(df_train)
    df_valid = format_text(df_valid)
    
    return df_train, df_valid

df = shuffle_data(df)
print(df.head())
# df_train, df_valid = fasttext_format(df)

# print(len(df_train.index))
# print(len(df_valid.index))

# df_train.to_csv('Data/news_titles.train', header=None, index=None, sep='\t')
# df_valid.to_csv('Data/news_titles.valid', header=None, index=None, sep='\t')


362282
             category                                       headlineDesc
68091           TASTE  This PB&J Bourbon Comes With A Side Of Childho...
4800         POLITICS  Rebuffing Yet Another Trump Attack, Sessions P...
186304      PARENTING  Spanking Alternatives: Experts' 8 Top Tips For...
30729           CRIME  Police Thwart Plot To Steal Enzo Ferrari’s Rem...
98021   ENTERTAINMENT  Humphrey Bogart and Clint Eastwood Still Make ...
        category                                       headlineDesc
0          TASTE  This PB&J Bourbon Comes With A Side Of Childho...
1       POLITICS  Rebuffing Yet Another Trump Attack, Sessions P...
2      PARENTING  Spanking Alternatives: Experts' 8 Top Tips For...
3          CRIME  Police Thwart Plot To Steal Enzo Ferrari’s Rem...
4  ENTERTAINMENT  Humphrey Bogart and Clint Eastwood Still Make ...


In [9]:
from torchtext.data import Field
text_field = Field(
    tokenize='basic_english', 
    lower=True
)
label_field = Field(sequential=False, use_vocab=False)
# sadly have to apply preprocess manually
preprocessed_text = df['headlineDesc'].apply(lambda x: text_field.preprocess(x))
# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d'
)
# get the vocab instance
vocab = text_field.vocab

In [10]:
# Load Fasttext Embeddings
from torchtext.vocab import FastText
embedding = FastText('simple')

In [11]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

20
1086


In [12]:
####### FOLLOWING CELLS INSPIRED BY PYTORCH OFFICIAL TURORIALS #######
####### https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html ######
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.8806,  0.6474, -0.2148, -0.8530,  0.5135]],
       grad_fn=<EmbeddingBackward>)


In [19]:
word_to_ix = vocab
print(len(vocab))
embeds = nn.Embedding(114813, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([vocab['the']], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

114813
tensor([[ 0.8517,  1.3011,  0.5214,  0.8494, -0.5577]],
       grad_fn=<EmbeddingBackward>)


In [20]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torchtext.legacy import data

# TEXT = data.Field(tokenize = 'spacy', lower = True)
# LABEL = data.LabelField()
TEXT = text_field
LABEL = label_field

news = data.TabularDataset(
    path='input/News_Category_Dataset_v2.json', format='json',
    fields={'headline': ('headline', TEXT),
            'short_description' : ('desc', TEXT),
             'category': ('category', LABEL)})

import random
SEED = 1234

trn, vld, tst = news.split(split_ratio=[0.7, 0.2, 0.1], random_state = random.seed(SEED))

TEXT.build_vocab(trn, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(trn)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (trn, vld, tst), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key= lambda x: len(x.headline), 
    sort_within_batch= False
    )


ValueError: Requested tokenizer <torchtext.vocab.FastText object at 0x000001BFB16C2070>, valid choices are a callable that takes a single string as input, "revtok" for the revtok reversible tokenizer, "subword" for the revtok caps-aware tokenizer, "spacy" for the SpaCy English tokenizer, or "moses" for the NLTK port of the Moses tokenization script.

In [ ]:
model_param = ModelParam(
    param_dict=dict(
        vocab_size=len(text_field.vocab),
        input_size=5
    )
)